In [24]:
import os,os.path as osp
import io,gzip
import functools as ft,itertools as it
import pycurl
import numpy as np,pandas as pd

In [3]:
parent_dir = osp.abspath(osp.join(os.getcwd(),os.pardir))
cfg = osp.join(parent_dir,'genelinks.cfg')
osp.exists(cfg)

True

In [33]:
D = {}
with open(cfg,'r') as f:
    lines = [l.strip() for l in f.readlines() if l]
fn = lambda X: X.split('.')[-2]
for k,g in it.groupby(sorted(lines,key=fn),key=fn):
    D[k] = list(g)
D.keys()

dict_keys(['23andme', '23andme-exome-vcf', 'IYG', 'ancestry', 'decodeme', 'ftdna-illumina', 'genes-for-good'])

In [39]:
txt = ['23andme','23andme-exome-vcf','ancestry','genes-for-good']
gzd = ['IYG','decodeme','ftdna-illumina']

In [183]:
def read_23andme(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = '\t'
    pg = io.StringIO(pagebytes.decode('ascii'))
    args = {
        'comment':'#','sep':sep,'dtype':object,'names':cols
    }
    return pd.read_csv(pg,**args)
def read_ancestry(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = ','
    args = {
        'skiprows':1,'sep':sep,'dtype':object,'names':cols
    }
    filt = lambda Y: not Y.startswith('#')
    decd = lambda X: X.replace('\t',',',3).replace('\t','')
    junk = pagebytes.decode('ascii')
    fpg = filter(filt,junk.splitlines())
    junk = [decd(x) for x in fpg]
    junk = io.StringIO('\n'.join(junk))
    return pd.read_csv(junk,**args)

In [184]:
def grab_link(L,mode='23andme'):
    C = pycurl.Curl()
    C.setopt(pycurl.MAXREDIRS,50)
    buff = io.BytesIO()
    f1 = lambda Y: Y and not Y.startswith('#')
    f2 = lambda X: '-'.join(X.split('/')[-1].split('.')[::2])
    rows = {}
    if mode=='23andme':
        to_df = read_23andme
    elif mode=='ancestry':
        to_df = read_ancestry
    for l in L:
        C.setopt(pycurl.URL,l)
        C.setopt(pycurl.WRITEDATA, buff)
        #C.setopt(pycurl.WRITEFUNCTION, buff.write)
        C.perform()
        genes = to_df(buff.getvalue())
        #_genes = io.StringIO(buff.getvalue().decode('ascii'))
        #genes = filter(f1,_genes.splitlines())
        #genes = pd.read_csv(_genes,**args)
        rows[f2(l)] = genes #[x.strip() for x in genes]
        buff.truncate(0)
    C.close()
    return rows
#f5 = lambda X: X.decode('ascii').replace('\t',',',3).replace('\t','')
#with open('496.data','rb') as f:


In [175]:
genes = grab_link(D['23andme'][:2])

In [164]:
genes['1002-497'].head(2)

,rsid,chrom,pos,alleles
0,rs4477212,1,82154,AA
1,rs3094315,1,752566,AG


In [185]:
tmp = grab_link(D['ancestry'][:1],'ancestry')['1001-496']

In [189]:
tmp.shape

(701478, 4)

In [188]:
len(D['ancestry'])+len(D['23andme'])

4036

In [194]:
print('{:,}'.format(4036*701478))
print('{:,}'.format(4036*701478//8))

2,831,165,208
353,895,651
